In [ ]:
!pip install flair==0.4.2
!pip install torch==1.1.0



In [ ]:
import pandas as pd
import os
import glob

In [ ]:
def load_data(file):
    fadir = '../input/urdufake2021/Training DatasetFIRE2021/Training Dataset@FIRE2021/'+file+'/Fake'
    redir = '../input/urdufake2021/Training DatasetFIRE2021/Training Dataset@FIRE2021/'+file+'/Real'


    txt_fake = os.path.join(fadir, '*.txt')
    txt_real = os.path.join(redir, '*.txt')

    fake=[]
    real=[]
    for txt_file in glob.glob(txt_fake):
        txt=open(txt_file,'r').read().replace('\n','')
        fake.append(txt[:500])
    for txt_file in glob.glob(txt_real):
        txt=open(txt_file,'r').read().replace('\n','')
        real.append(txt[:500])
    
    fdf=pd.DataFrame()
    fdf['news']=fake
    fdf['label']='fake'

    rdf=pd.DataFrame()
    rdf['news']=real
    rdf['label']='real'
    data=pd.DataFrame()
    
    news=list(fdf['news'])+list(rdf['news'])
    lbl=list(fdf['label'])+list(rdf['label'])
    data['news']=news
    data['label']=lbl
    return data

train=load_data('Train')
dev=load_data('Test')

In [ ]:
train.head()

In [ ]:
data1 = pd.DataFrame()
data1['label']=train.label
data1['text']=train.news
data1.head()

In [ ]:
 
data1['label'] = '__label__' + data1['label'].astype(str)
data1.to_csv('train.txt', sep='\t', index = False, header = False)


In [ ]:
test1 = pd.DataFrame()
test1['label']=dev.label
test1['text']=dev.news
test1.head()

In [ ]:
 
test1['label'] = '__label__' + test1['label'].astype(str)
test1.to_csv('dev.txt', sep='\t', index = False, header = False)


In [ ]:
import argparse
from typing import Tuple
from pathlib import Path


def trainer(file_path: Path,
            filenames: Tuple[str, str, str],
            checkpoint: str,
            stack: str,
            n_epochs: int) -> None:
    """Train sentiment model using Flair NLP library:
    https://github.com/zalandoresearch/flair/blob/master/resources/docs/TUTORIAL_7_TRAINING_A_MODEL.md
    To help provide added context, we can stack Glove, Bert or ELMo embeddings along with Flair embeddings.
    """
        # pip install flair allennlp
    from flair.datasets import ClassificationCorpus
    from flair.embeddings import FlairEmbeddings, DocumentRNNEmbeddings
    from flair.models import TextClassifier
    from flair.trainers import ModelTrainer
    from flair.training_utils import EvaluationMetric
    from flair.visual.training_curves import Plotter

    if stack == "glove":
        from flair.embeddings import WordEmbeddings
        stacked_embedding = WordEmbeddings('glove')
    elif stack == "elmo":
        from flair.embeddings import ELMoEmbeddings
        stacked_embedding = ELMoEmbeddings('original')
    elif stack == "bert":
        from flair.embeddings import BertEmbeddings
        stacked_embedding = BertEmbeddings('bert-base-multilingual-cased')
    else:
        stacked_embedding = None

    # Define and Load corpus from the provided dataset
    train, dev, test = filenames
    corpus = ClassificationCorpus(
        file_path,
        train_file=train,
        dev_file=dev,
        test_file=test,
    )
    # Create label dictionary from provided labels in data
    label_dict = corpus.make_label_dictionary()

    # Stack Flair string-embeddings with optional embeddings
    word_embeddings = list(filter(None, [
        stacked_embedding
    ]))
    # Initialize document embedding by passing list of word embeddings
    document_embeddings = DocumentRNNEmbeddings(
        word_embeddings,
        hidden_size=512,
        reproject_words=True,
        reproject_words_dimension=256,
    )
    # Define classifier
    classifier = TextClassifier(
        document_embeddings,
        label_dictionary=label_dict,
        multi_label=False
    )

    if not checkpoint:
        trainer = ModelTrainer(classifier, corpus)
    else:
        # If checkpoint file is defined, resume training
        checkpoint = classifier.load_checkpoint(Path(checkpoint))
        trainer = ModelTrainer.load_from_checkpoint(checkpoint, corpus)

    # Begin training (enable checkpointing to continue training at a later time, if desired)
    trainer.train(
        file_path,
        EvaluationMetric.MACRO_F1_SCORE,
        max_epochs=n_epochs,
        checkpoint=True
    )

    # Plot curves and store weights and losses
    plotter = Plotter()
    plotter.plot_training_curves(file_path / 'loss.tsv')
    plotter.plot_weights(file_path / 'weights.txt')



In [ ]:
#!mkdir './ckp'

In [ ]:
filepath = Path('./')
filenames = ('train.txt', 'dev.txt', 'dev.txt')
trainer(filepath, filenames, None, stack='bert', n_epochs=5)



In [2]:
import os
import numpy
import pandas as pd
import re
import glob
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk import trigrams
from nltk import bigrams
from nltk import ngrams
from nltk.collections import defaultdict
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.linear_model import LogisticRegression
from gensim.models import KeyedVectors
from collections import Counter 
from nltk.classify.util import accuracy
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report,accuracy_score
import warnings
import pandas as pd
import nltk
import gensim
import collections
warnings.filterwarnings('ignore')

In [14]:
def load_data(file):
    fadir = '../input/urdufake2021/Training DatasetFIRE2021/Training Dataset@FIRE2021/'+file+'/Fake'
    redir = '../input/urdufake2021/Training DatasetFIRE2021/Training Dataset@FIRE2021/'+file+'/Real'


    txt_fake = os.path.join(fadir, '*.txt')
    txt_real = os.path.join(redir, '*.txt')

    fake=[]
    real=[]
    for txt_file in glob.glob(txt_fake):
        txt=open(txt_file,'r').read().replace('\n','')
        fake.append(txt)
    for txt_file in glob.glob(txt_real):
        txt=open(txt_file,'r').read().replace('\n','')
        real.append(txt)
    
    fdf=pd.DataFrame()
    fdf['news']=fake
    fdf['label']='F'

    rdf=pd.DataFrame()
    rdf['news']=real
    rdf['label']='R'
    data=pd.DataFrame()
    
    news=list(fdf['news'])+list(rdf['news'])
    lbl=list(fdf['label'])+list(rdf['label'])
    data['news']=news
    data['label']=lbl
    return data

data=load_data('Train')
test=load_data('Test')

In [15]:
def clean_text(text):
    stopwords=[]#open('/home/fazl/Desktop/FIRE2020/Urdu/urdu_stopwords.txt').read().split('\n')
    unwanted=['ـ','_','"','！',"'",'=','@','&','%','.',',',':','\\','$','^','<','>','!','?','{','}',';','\n','\t','(',')','[',']','/','*','+','#','\u200c','\ufeff','-','_','|']
    
    for i in unwanted:
        text = text.replace(i,'')
    sent=[]
    text=text.split(' ')
    for word in text:
        if word not in stopwords and len(word)>=2:
            sent.append(word)
    return ' '.join(sent).strip()

data['news']=data['news'].apply(lambda x:clean_text(x))
test['news']=test['news'].apply(lambda x:clean_text(x))


In [16]:
def load_test(path):
    txt_files = glob.glob(path+'/*.txt')
    #print('files :', txt_files)

    data=[]
    ids=[]
    for jf in txt_files:
        file=open(jf).read().replace('\n', ' ')
        data.append(clean_text(file))
        ids.append(jf.split('/')[-1].split('.')[0])

            
    return data,ids
    

submission=pd.DataFrame()
txxt,ids=load_test('../input/urdufake2021/Test Dataset  FIRE 2021/Test Dataset/')
submission['text']=txxt
submission['File No']=ids
print(len(submission))
submission.head()

In [17]:
from sklearn import model_selection, preprocessing, metrics, svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import decomposition, ensemble

import pandas as pd
import numpy as np

import string

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import xgboost
#from keras.preprocessing import text, sequence
#from keras import layers, models, optimizers
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,HashingVectorizer



In [18]:
sentences=list(data.news)
from scipy.sparse import hstack
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    ngram_range=(1, 3),
    max_features=30000)
word_vectorizer.fit(sentences)

char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(2, 5),
    max_features=30000)
char_vectorizer.fit(sentences)


'''sn_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer=get_sngrams,
    
    max_features=40000)
sn_vectorizer.fit(sentences)'''


train_word_features = word_vectorizer.transform(data.news)
test_word_features = word_vectorizer.transform(test.news)

train_char_features = char_vectorizer.transform(data.news)
test_char_features = char_vectorizer.transform(test.news)

'''train_sn_features = sn_vectorizer.transform(train_en.cleaned)
test_sn_features = sn_vectorizer.transform(test_en.cleaned)'''


x_train = hstack([train_char_features, train_word_features])#,train_sn_features])
x_test = hstack([test_char_features, test_word_features])#,test_sn_features])

y_train=data.label
y_test=test.label

In [19]:
sub_word_features = word_vectorizer.transform(submission.text)
sub_char_features = char_vectorizer.transform(submission.text)
x_sub = hstack([sub_char_features, sub_word_features])#,test_sn_features])


In [20]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC,SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score


lr=LogisticRegression(C=20)
lr.fit(x_train, y_train)
y_pred = lr.predict(x_test) 
print(classification_report(y_test, y_pred,digits=4))

In [22]:
s_pred = lr.predict(x_sub)
submission['Real/Fake']=s_pred
print(submission['Real/Fake'].value_counts())
sub1=submission.drop('text',1)
submission=submission.drop('Real/Fake',1)
print(sub1.head())
sub1.to_csv('NoFS_LR_0.7881.csv',index=None)
submission.head()

In [23]:
svm_clf=SVC(kernel='linear',probability=True)
svm_clf.fit(x_train, y_train)
y_pred = svm_clf.predict(x_test)
print(classification_report(y_test, y_pred,digits=4))

In [24]:
s_pred = svm_clf.predict(x_sub)
submission['Real/Fake']=s_pred
print(submission['Real/Fake'].value_counts())
sub1=submission.drop('text',1)
submission=submission.drop('Real/Fake',1)
print(sub1.head())
sub1.to_csv('NoFS_LSVM_0.7990.csv',index=None)
submission.head()

In [25]:
from sklearn.ensemble import RandomForestClassifier

model=RandomForestClassifier(n_estimators=1000)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print(classification_report(y_test, y_pred,digits=4))

In [26]:
s_pred = model.predict(x_sub)
submission['Real/Fake']=s_pred
print(submission['Real/Fake'].value_counts())
sub1=submission.drop('text',1)
submission=submission.drop('Real/Fake',1)
print(sub1.head())
sub1.to_csv('NoFS_RFC_0.7592.csv',index=None)
submission.head()

In [27]:
from sklearn.neural_network import MLPClassifier
mlp=MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter=300,activation = 'relu',solver='adam',random_state=1)# Again, dual = True. Default solver = 'liblinear'. It's recommended for smaller databases. For bigger databases, 'saga' could be used.
mlp.fit(x_train, y_train)
y_pred = mlp.predict(x_test)
print(classification_report(y_test, y_pred,digits=4))

In [28]:
s_pred = mlp.predict(x_sub)
submission['Real/Fake']=s_pred
print(submission['Real/Fake'].value_counts())
sub1=submission.drop('text',1)
submission=submission.drop('Real/Fake',1)
print(sub1.head())
sub1.to_csv('NoFS_mlp_0.7734.csv',index=None)
submission.head()

In [29]:
import xgboost as xgb
xgb_clf=xgb.XGBClassifier(max_depth=20, n_estimators=80, learning_rate=0.1, colsample_bytree=.7, gamma=.01, reg_alpha=4, objective='multi:softmax',num_class=2)
xgb_clf.fit(x_train, y_train)
y_pred = xgb_clf.predict(x_test)
print(classification_report(y_test, y_pred,digits=4))

In [30]:
s_pred = xgb_clf.predict(x_sub)
submission['Real/Fake']=s_pred
print(submission['Real/Fake'].value_counts())
sub1=submission.drop('text',1)
submission=submission.drop('Real/Fake',1)
print(sub1.head())
sub1.to_csv('NoFS_xgb_0.7858.csv',index=None)
submission.head()

In [31]:
from sklearn.ensemble import VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC,SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.multiclass import OneVsRestClassifier
import xgboost as xgb
from sklearn.linear_model import SGDClassifier
m1=LogisticRegression(C=20)
m2=SVC(kernel='linear',probability=True)
m3=RandomForestClassifier(n_estimators=1000)
m4=MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter=300,activation = 'relu',solver='adam',random_state=1)
m5=xgb_clf=xgb.XGBClassifier(max_depth=20, n_estimators=80, learning_rate=0.1, colsample_bytree=.7, gamma=.01, reg_alpha=4, objective='multi:softmax',num_class=2)

In [32]:
estimators=[('lr', m1), ('lsvm', m2),('rfc', m4),('mlp', m4),('xgb', m5)]
clf = VotingClassifier(estimators, voting='soft')
clf.fit(x_train, y_train)

y_pred = clf.predict(x_test) 
print(classification_report(y_test, y_pred,digits=4))

In [33]:
s_pred = clf.predict(x_sub)
submission['Real/Fake']=s_pred
print(submission['Real/Fake'].value_counts())
sub1=submission.drop('text',1)
submission=submission.drop('Real/Fake',1)
print(sub1.head())
sub1.to_csv('NoFS_soft_ensemble.csv',index=None)
submission.head()

In [34]:
estimators=[('lr', m1), ('lsvm', m2),('rfc', m4),('mlp', m4),('xgb', m5)]
clf = VotingClassifier(estimators, voting='hard')
clf.fit(x_train, y_train)

y_pred = clf.predict(x_test) 
print(classification_report(y_test, y_pred,digits=4))

In [35]:
s_pred = clf.predict(x_sub)
submission['Real/Fake']=s_pred
print(submission['Real/Fake'].value_counts())
sub1=submission.drop('text',1)
submission=submission.drop('Real/Fake',1)
print(sub1.head())
sub1.to_csv('NoFS_hard_ensemble.csv',index=None)
submission.head()

In [36]:
Xw=train_word_features
Xw=Xw.toarray()

In [37]:
df1=pd.DataFrame(Xw,columns=word_vectorizer.get_feature_names())
df1.head()

In [38]:
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import f_classif
import argparse
import os  
import codecs
from collections import defaultdict
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer  
import pickle
from sklearn import preprocessing
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC 
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import f_classif
import pandas as pd
import numpy
import scipy.sparse

#definig variables
y = y_train
features=df1.columns
feat=[]
chi2_val=[]
mi_val=[]
f_val=[]
f_size=15000

#Feature selection method using classifiers
c=chi2(Xw,y)
mi=mutual_info_classif(Xw, y, discrete_features='auto', n_neighbors=4, copy=True, random_state=None)
f1=f_classif(Xw,y)

#converting FS to Dataframe
c=pd.Series(c[0], index=df1.columns)
mi=pd.Series(mi,index=df1.columns)
f=pd.Series(f1[0],index=df1.columns)


#Assining coressponding feature to its values
for i in range(len(features)):
    feat.append(features[i])
    chi2_val.append(c[i])
    mi_val.append(mi[i])
    f_val.append(f[i])
    
#Arranging in assending order    
dic={'features':features,'chi_2_values':chi2_val}
chi2_tab=pd.DataFrame(dic)
chi2_tab.sort_values('chi_2_values', inplace=True, ascending=False)
chi2_tab_rl=chi2_tab.head(f_size)

dic1={'features':features,'mi_values':mi_val}
mi_tab=pd.DataFrame(dic1)
mi_tab.sort_values('mi_values', inplace=True, ascending=False)
mi_tab_rl=mi_tab.head(f_size)


dic2={'features':features,'f_values':f_val}
f_tab=pd.DataFrame(dic2)
f_tab.sort_values('f_values', inplace=True, ascending=False)
f_tab_rl=f_tab.head(f_size)

#selecting ranklist
chi2_rl=list(chi2_tab_rl['features'])
mi_rl=list(mi_tab_rl['features'])
f_rl=list(f_tab_rl['features'])
rank_list=chi2_rl+mi_rl+f_rl


rl={}
for i in range(len(rank_list)):
    for j in df1.columns:
        if  rank_list[i]==j:
            rl.__setitem__(rank_list[i],df1[j])
rl_tf_idf=pd.DataFrame(rl)
rl_tf_idf


In [39]:
wordNew=rl_tf_idf

In [40]:
Xc=train_char_features
Xc=Xc.toarray()
df2=pd.DataFrame(Xc,columns=char_vectorizer.get_feature_names())
df2.head()

In [41]:
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import f_classif
import argparse
import os  
import codecs
from collections import defaultdict
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer  
import pickle
from sklearn import preprocessing
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC 
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import f_classif
import pandas as pd
import numpy
import scipy.sparse

#definig variables
y = y_train
features=df2.columns
feat=[]
chi2_val=[]
mi_val=[]
f_val=[]
f_size=15000

#Feature selection method using classifiers
c=chi2(Xc,y)
mi=mutual_info_classif(Xc, y, discrete_features='auto', n_neighbors=4, copy=True, random_state=None)
f1=f_classif(Xc,y)

#converting FS to Dataframe
c=pd.Series(c[0], index=df2.columns)
mi=pd.Series(mi,index=df2.columns)
f=pd.Series(f1[0],index=df2.columns)


#Assining coressponding feature to its values
for i in range(len(features)):
    feat.append(features[i])
    chi2_val.append(c[i])
    mi_val.append(mi[i])
    f_val.append(f[i])
    
#Arranging in assending order    
dic={'features':features,'chi_2_values':chi2_val}
chi2_tab=pd.DataFrame(dic)
chi2_tab.sort_values('chi_2_values', inplace=True, ascending=False)
chi2_tab_rl=chi2_tab.head(f_size)

dic1={'features':features,'mi_values':mi_val}
mi_tab=pd.DataFrame(dic1)
mi_tab.sort_values('mi_values', inplace=True, ascending=False)
mi_tab_rl=mi_tab.head(f_size)


dic2={'features':features,'f_values':f_val}
f_tab=pd.DataFrame(dic2)
f_tab.sort_values('f_values', inplace=True, ascending=False)
f_tab_rl=f_tab.head(f_size)

#selecting ranklist
chi2_rl=list(chi2_tab_rl['features'])
mi_rl=list(mi_tab_rl['features'])
f_rl=list(f_tab_rl['features'])
rank_list=chi2_rl+mi_rl+f_rl


rl={}
for i in range(len(rank_list)):
    for j in df2.columns:
        if  rank_list[i]==j:
            rl.__setitem__(rank_list[i],df2[j])
rl_tf_idf=pd.DataFrame(rl)
rl_tf_idf


In [42]:
charNew=rl_tf_idf

In [43]:
sentences=list(data.news)
from scipy.sparse import hstack
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    ngram_range=(1, 3),
    vocabulary=wordNew)
word_vectorizer.fit(sentences)

char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(2, 5),
    vocabulary=charNew)
char_vectorizer.fit(sentences)


'''sn_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer=get_sngrams,
    
    max_features=40000)
sn_vectorizer.fit(sentences)'''


train_word_features = word_vectorizer.transform(data.news)
test_word_features = word_vectorizer.transform(test.news)

train_char_features = char_vectorizer.transform(data.news)
test_char_features = char_vectorizer.transform(test.news)

'''train_sn_features = sn_vectorizer.transform(train_en.cleaned)
test_sn_features = sn_vectorizer.transform(test_en.cleaned)'''


x_train = hstack([train_char_features, train_word_features])#,train_sn_features])
x_test = hstack([test_char_features, test_word_features])#,test_sn_features])

y_train=data.label
y_test=test.label

In [44]:
sub_word_features = word_vectorizer.transform(submission.text)
sub_char_features = char_vectorizer.transform(submission.text)
x_sub = hstack([sub_char_features, sub_word_features])#,test_sn_features])


In [45]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC,SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score


lr=LogisticRegression(C=20)
lr.fit(x_train, y_train)
y_pred = lr.predict(x_test) 
print(classification_report(y_test, y_pred,digits=4))

In [46]:
s_pred = lr.predict(x_sub)
submission['Real/Fake']=s_pred
print(submission['Real/Fake'].value_counts())
sub1=submission.drop('text',1)
submission=submission.drop('Real/Fake',1)
print(sub1.head())
sub1.to_csv('FS_LR_0.8007.csv',index=None)
submission.head()

In [47]:
svm_clf=SVC(kernel='linear',probability=True)
svm_clf.fit(x_train, y_train)
y_pred = svm_clf.predict(x_test)
print(classification_report(y_test, y_pred,digits=4))

In [48]:
s_pred = svm_clf.predict(x_sub)
submission['Real/Fake']=s_pred
print(submission['Real/Fake'].value_counts())
sub1=submission.drop('text',1)
submission=submission.drop('Real/Fake',1)
print(sub1.head())
sub1.to_csv('FS_LSVM_0.7971.csv',index=None)
submission.head()

In [49]:
from sklearn.ensemble import RandomForestClassifier

model=RandomForestClassifier(n_estimators=1000)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print(classification_report(y_test, y_pred,digits=4))

In [50]:
s_pred = model.predict(x_sub)
submission['Real/Fake']=s_pred
print(submission['Real/Fake'].value_counts())
sub1=submission.drop('text',1)
submission=submission.drop('Real/Fake',1)
print(sub1.head())
sub1.to_csv('FS_RFC_0.7440.csv',index=None)
submission.head()

In [51]:
from sklearn.neural_network import MLPClassifier
mlp=MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter=300,activation = 'relu',solver='adam',random_state=1)# Again, dual = True. Default solver = 'liblinear'. It's recommended for smaller databases. For bigger databases, 'saga' could be used.
mlp.fit(x_train, y_train)
y_pred = mlp.predict(x_test)
print(classification_report(y_test, y_pred,digits=4))

In [52]:
s_pred = mlp.predict(x_sub)
submission['Real/Fake']=s_pred
print(submission['Real/Fake'].value_counts())
sub1=submission.drop('text',1)
submission=submission.drop('Real/Fake',1)
print(sub1.head())
sub1.to_csv('FS_mlp_0.7641.csv',index=None)
submission.head()

In [53]:
import xgboost as xgb
xgb_clf=xgb.XGBClassifier(max_depth=20, n_estimators=80, learning_rate=0.1, colsample_bytree=.7, gamma=.01, reg_alpha=4, objective='multi:softmax',num_class=2)
xgb_clf.fit(x_train, y_train)
y_pred = xgb_clf.predict(x_test)
print(classification_report(y_test, y_pred,digits=4))

In [54]:
s_pred = xgb_clf.predict(x_sub)
submission['Real/Fake']=s_pred
print(submission['Real/Fake'].value_counts())
sub1=submission.drop('text',1)
submission=submission.drop('Real/Fake',1)
print(sub1.head())
sub1.to_csv('FS_xgb_0.7931.csv',index=None)
submission.head()

In [55]:
estimators=[('lr', m1), ('lsvm', m2),('rfc', m4),('mlp', m4),('xgb', m5)]
clf = VotingClassifier(estimators, voting='soft')
clf.fit(x_train, y_train)

y_pred = clf.predict(x_test) 
print(classification_report(y_test, y_pred,digits=4))

In [56]:
s_pred = clf.predict(x_sub)
submission['Real/Fake']=s_pred
print(submission['Real/Fake'].value_counts())
sub1=submission.drop('text',1)
submission=submission.drop('Real/Fake',1)
print(sub1.head())
sub1.to_csv('FS_soft_ensemble.csv',index=None)
submission.head()

In [57]:
estimators=[('lr', m1), ('lsvm', m2),('rfc', m4),('mlp', m4),('xgb', m5)]
clf = VotingClassifier(estimators, voting='hard')
clf.fit(x_train, y_train)

y_pred = clf.predict(x_test) 
print(classification_report(y_test, y_pred,digits=4))

In [58]:
s_pred = clf.predict(x_sub)
submission['Real/Fake']=s_pred
print(submission['Real/Fake'].value_counts())
sub1=submission.drop('text',1)
submission=submission.drop('Real/Fake',1)
print(sub1.head())
sub1.to_csv('FS_hard_ensemble.csv',index=None)
submission.head()